# Natural Language Processing

## Exercise Sheet 8

In [2]:
#imports for all exercises
import nltk

### Exercise 1

Write a recursive function to traverse a tree and return the depth of the tree, such that a tree with a single node would have depth zero. (Hint: the depth of a subtree is the maximum depth of its children, plus one.)
Test your function with the two trees produced by the `ChartParser` for the `groucho_grammar` and the sentence "I shot an elephant in my pajamas". The result can be verified with the `Tree.height()` function.



In [53]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)
for tree in parser.parse(sent):
    print(tree.height())

6
7


In [56]:
def traverse(tree):
    try:
        tree.label()
    except: 
        return 0
    max_child_height = 1
    for child in tree:
            max_child_height = max(max_child_height, traverse(child))
    return 1 + max_child_height

In [57]:
for tree in parser.parse(sent):
    result = traverse(tree)
    print(result)


6
7


In [58]:
# This version does not work

def traverse(tree,depth=0):
    try:
        tree.label()
    except AttributeError:
        return depth
    else:
        try:
            return  max(traverse(tree[0],depth +1),traverse(tree[1], depth+1))
        except:
            return  max(traverse(tree[0],depth+1),0)

        


for tree in parser.parse(sent):
    result = traverse(tree)
    print(result)


5
4


In [24]:
def traverse(tree):
    ret = 0
    try:
        print(tree.label())
        tree.label()
    except:
        return ret + 1
    else:
        for child in tree:
            return ret + traverse(child)
        return ret +1


for tree in parser.parse(sent):
    result = traverse(tree)
    print(result)

S
NP
4
S
NP
4


### Exercise 2

Write a recursive function `bracketing(tree)` that produces a nested bracketing for a `tree`, leaving out the leaf nodes, and displaying the non-terminal labels after their subtrees. Consecutive categories should be separated by space. Test your function with the tree: 

In [61]:
from nltk.corpus import treebank
t = treebank.parsed_sents('wsj_0001.mrg')[0]
print(t)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))


In [143]:
def bracketing(tree,string=""):
    try:
        tree.label()
    except: 
        return string
    string += "["
    firstIt = False
    for child in tree:
        if firstIt:
            string += " "
        string += bracketing(child)
        firstIt = True
    firstIt = False
    string += "]"
    string += tree.label() 
    return string.replace("[]","")

In [144]:

result = bracketing(t)
print(result)

[[[NNP NNP]NP , [[CD NNS]NP JJ]ADJP ,]NP-SBJ [MD [VB [DT NN]NP [IN [DT JJ NN]NP]PP-CLR [NNP CD]NP-TMP]VP]VP .]S


In [88]:
def bracketing(t):
    try:
        t.label()
    except AttributeError:
        print(t, end=" ")
    else:
        # Now we know that t.node is defined
        print('(', t.label(), end=" ")
        for child in t:
            traverse(child)
        print(')', end=" ")
bracketing(t)

( S ) 

In [5]:
# bracketing(t)

    [[[NNP NNP]NP , [[CD NNS]NP JJ]ADJP ,]NP-SBJ [MD [VB [DT NN]NP 
    [IN [DT JJ NN]NP]PP-CLR [NNP CD]NP-TMP]VP]VP .]S

### Exercise 3

Modify the functions `init_wfst()` and `complete_wfst()` so that the contents of each cell in the WFST is a set of non-terminal symbols rather than a single non-terminal. Test your function with the `groucho_grammar` and the sentence "I shot an elephant in my pajamas". 

In [183]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        # print(productions)
        wfst[i][i+1] = productions[0].lhs()
    return wfst

def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                nt1, nt2 = wfst[start][mid], wfst[mid][end]
                if nt1 and nt2 and (nt1,nt2) in index:
                    wfst[start][end] = index[(nt1,nt2)]
                    if trace:
                        print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                        (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
    return wfst

def display(wfst):
    print('\nWFST ' + ' '.join(("%-4d" % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d   " % i, end=" ")
        for j in range(1, len(wfst)):
            print("%-4s" % (wfst[i][j] or '.'), end=" ")
        print()

wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
# wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
# display(wfst)

[N -> 'elephant']
[N -> 'elephant']
[N -> 'elephant']
[N -> 'elephant']
[N -> 'elephant']
[N -> 'elephant']
[N -> 'elephant']

WFST 1    2    3    4    5    6    7   
0    NP   .    .    .    .    .    .    
1    .    V    .    .    .    .    .    
2    .    .    Det  .    .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    .    
5    .    .    .    .    .    Det  .    
6    .    .    .    .    .    .    N    


Change the line:

    NP -> Det N | Det N PP | 'I' 

in `groucho_grammar` to:

    NP -> Det N | NP PP | 'I' 

to verify in the trace of `complete_wfst()` that there are now two lines for `cell(1,7)`:

    [1]   V [2]  NP [7] ==> [1]  VP [7]
    [1]  VP [4]  PP [7] ==> [1]  VP [7]

In [146]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | NP PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")
wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
display(wfst)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    .    .    .    .    
1    .    V    .    .    .    .    .    
2    .    .    Det  .    .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    .    
5    .    .    .    .    .    Det  .    
6    .    .    .    .    .    .    N    
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[2]  NP [4]  PP [7] ==> [2]  NP [7]
[1]   V [2]  NP [7] ==> [1]  VP [7]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]

WFST 1    2    3    4    5    6    7   
0    NP   .    .    S    .    .    S    
1    .    V    .    VP   .    .    VP   
2    .    .    Det  NP   .    .    NP   
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    PP   
5    .    .    .    .    .    Det  NP   
6    .    .    .    .    .    .    N    


Change the line:

    VP -> V NP | VP PP

in `groucho_grammar` to: 

    VP -> V NP
    VPC -> VP PP 

and check that `cell(1,7)` now contains `{VPC, VP}`.

In [147]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | NP PP | 'I'
VP -> V NP 
VPC -> VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")
wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
display(wfst)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    .    .    .    .    
1    .    V    .    .    .    .    .    
2    .    .    Det  .    .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    .    
5    .    .    .    .    .    Det  .    
6    .    .    .    .    .    .    N    
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[2]  NP [4]  PP [7] ==> [2]  NP [7]
[1]   V [2]  NP [7] ==> [1]  VP [7]
[1]  VP [4]  PP [7] ==> [1] VPC [7]

WFST 1    2    3    4    5    6    7   
0    NP   .    .    S    .    .    .    
1    .    V    .    VP   .    .    VPC  
2    .    .    Det  NP   .    .    NP   
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    PP   
5    .    .    .    .    .    Det  NP   
6    .    .    .    .    .    .    N    


Finally, change the line:

    S -> NP VP

in `groucho_grammar` to:

    S -> NP VP | NP VPC

and check that now there are two lines in the trace of `complete_wfst()` for the `cell(0,7)`:

    [0]  NP [1] VPC [7] ==> [0]   S [7]
    [0]  NP [1]  VP [7] ==> [0]   S [7]

In [148]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP | NP VP
PP -> P NP
NP -> Det N | NP PP | 'I'
VP -> V NP 
VPC -> VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")
wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
display(wfst)


WFST 1    2    3    4    5    6    7   
0    NP   .    .    .    .    .    .    
1    .    V    .    .    .    .    .    
2    .    .    Det  .    .    .    .    
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    .    
5    .    .    .    .    .    Det  .    
6    .    .    .    .    .    .    N    
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[2]  NP [4]  PP [7] ==> [2]  NP [7]
[1]   V [2]  NP [7] ==> [1]  VP [7]
[1]  VP [4]  PP [7] ==> [1] VPC [7]

WFST 1    2    3    4    5    6    7   
0    NP   .    .    S    .    .    .    
1    .    V    .    VP   .    .    VPC  
2    .    .    Det  NP   .    .    NP   
3    .    .    .    N    .    .    .    
4    .    .    .    .    P    .    PP   
5    .    .    .    .    .    Det  NP   
6    .    .    .    .    .    .    N    


### Exercise 4

Modify the function `complete_wfst()` from Exercise 3 so that when a non-terminal symbol is added to a cell in the WFST, the content of the variable `mid` is also added, i.e. we add a tuple `(symbol, mid)`. In `init_wfst()`, use `(symbol, i+1)` instead. Change also the function `display()` accordingly. Test your implementation with the final grammar from Exercise 3 and the sentence "I shot an elephant in my pajamas". 

In [230]:
#I comleted the exercise, the results vary from the solution as i did not finish exercise 3

groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")
sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        # print(productions)
        wfst[i][i+1] = (productions[0].lhs(),i+1)
    return wfst

def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                try:
                    nt1, nt2 = wfst[start][mid][0], wfst[mid][end][0]
                    if nt1 and nt2 and (nt1,nt2) in index:
                        wfst[start][end] = (index[(nt1,nt2)],str(mid))
                        if trace:
                            print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                            (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
                except:
                    ""
    return wfst

def display(wfst):
    print('\nWFST ' + '            '.join(("%-4d" % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d   " % i, end=" ")
        for j in range(1, len(wfst)):
            if wfst[i][j] == None:
                print("%-4s" % (wfst[i][j] or '.'), end="            ")
            else:
                end = ""
                diff = 16 - len(str(wfst[i][j]))
                for x in range(diff):
                    end += " "
                print("%-4s" % (str(wfst[i][j]) or '.'), end=end)
        print()

wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
display(wfst)


WFST 1               2               3               4               5               6               7   
0    (NP, 1)         .               .               .               .               .               .               
1    .               (V, 2)          .               .               .               .               .               
2    .               .               (Det, 3)        .               .               .               .               
3    .               .               .               (N, 4)          .               .               .               
4    .               .               .               .               (P, 5)          .               .               
5    .               .               .               .               .               (Det, 6)        .               
6    .               .               .               .               .               .               (N, 7)          
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP

It should produce the following output:

    WFST      1          2          3          4          5          6          7         
    0         {(NP, 1)}  .          .          {(S, 1)}   .          .          {(S, 1)}   
    1         .          {(V, 2)}   .          {(VP, 2)}  .          .          {(VP, 2), (VPC, 4)} 
    2         .          .          {(Det, 3)} {(NP, 3)}  .          .          {(NP, 4)}  
    3         .          .          .          {(N, 4)}   .          .          .          
    4         .          .          .          .          {(P, 5)}   .          {(PP, 5)}  
    5         .          .          .          .          .          {(Det, 6)} {(NP, 6)}  
    6         .          .          .          .          .          .          {(N, 7)}    

### Exercise 5

Use the extended WFST from Exercise 4 to retrace the parse trees for our example sentence "I shot an elephant in my pajamas''. Write a recursive function `retrace(WFST, tokens)` (the second parameter `tokens` contains the token list \['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas'\] for our example sentence). Start with `cell(0,7)` (or `cell(0,len(tokens))` in general) and use the information in `mid` to follow the productions to `cell(0,mid)` and `cell(mid,7)`, and so on. If we reach a terminal symbol, i.e. a `cell(i,i+1)`, the corresponding token from `tokens` shall be displayed. 

The function should produce the following output:

    S    -> NP   -> I 
            VPC  -> VP   -> V    -> shot 
                            NP   -> Det  -> an 
                                    N    -> elephant 
                    PP   -> P    -> in 
                            NP   -> Det  -> my 
                                    N    -> pajamas 
            VP   -> V    -> shot 
                    NP   -> NP   -> Det  -> an 
                                    N    -> elephant 
                            PP   -> P    -> in 
                                    NP   -> Det  -> my 
                                            N    -> pajamas  

### Exercise 6

Process each tree of the Penn Treebank Corpus sample `nltk.corpus.treebank` and extract the productions with the help of `Tree.productions()`. Discard the productions that occur only once and those that are lexical (i.e. the right-hand side contains at least one terminal token). Productions with the same left-hand side can be collapsed using a dictionary with the left-hand sides as keys and sets of right-hand sides as values.

Print the value for the left-hand side 'NP' using the format: 

    DT JJS NN NN | DT VBG NN NN | DT NNP CD NN | DT NN NNS ...
